<a href="https://colab.research.google.com/github/LIMDANBI/Hackathon/blob/main/KoBERT_fine_tuning(%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%87%BC%ED%95%91_%EB%A6%AC%EB%B7%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 환경 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#https://github.com/SKTBrain/KoBERT 의 파일들을 Colab으로 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install transformers
!pip3 install kobert-transformers
!pip install gluonnlp
!pip install sentencepiece
!pip install mxnet
!pip install torch

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-xj2rqfqb/kobert-tokenizer_b8fc1575db73491eab4985ce784f3a8d
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-xj2rqfqb/kobert-tokenizer_b8fc1575db73491eab4985ce784f3a8d
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=0f2afdaba110bd27e6c8f143df812bc60c224341132b55f13610e5f149f79335
  Stored in directory: /tmp/pip-ephem-wheel-cache-ag3pwl8k/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.6 MB/s et

In [ ]:
# Transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# Setting Library
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

device = torch.device("cuda:0")

# 데이터 불러오기

In [ ]:
# -*- coding: cp949 -*-
data = pd.read_excel("/content/drive/MyDrive/Colab_Notebooks/NewsBigData/Labelled_KoBERT_Dataset_v1.xlsx")
data

,sentence,label
0,도널드 트럼프 미국 대통령이 30일 역대 처음으로 우리나라 대통령과 비무장지대(dm...,0
1,물 흐르듯 대화가 필요하다'고 작성했다..,0
2,"통상 기결수는 미결수들이 구금된 구치소가 아닌 일반 교도소에 구금되지만, 최씨의 경...",0
3,그리고 주권자 국민의 관심이 각별하게 필요한 시간'이라고 강조하며 국민들의 관심을 ...,0
4,"박 최고위원은 '독일이 2015년 최저임금제를 도입하고, 미국도 최저임금제를 도입하...",0
...,...,...
199995,신세계건설은 지난달 26일 사업목적을 기존의 건설업 및 건물관리 외에 복합쇼핑몰의 ...,1
199996,카메라 모듈의 ois 및 무선 충전 기능 채택으로 수익성 개선도 기대된다..,1
199997,"삼성전기의 투자의견 '매수'를 유지하고, 12개월 목표주가는 기존 8만5000원에서...",1
199998,"지난해는 춘절이 1월에 있어 지난 2013년 12월에 판매가 집중됐으나, 올해는 춘...",1


In [ ]:
data = data.drop_duplicates(['sentence']) # 중복 제거

In [ ]:
data

,sentence,label
0,도널드 트럼프 미국 대통령이 30일 역대 처음으로 우리나라 대통령과 비무장지대(dm...,0
1,물 흐르듯 대화가 필요하다'고 작성했다..,0
2,"통상 기결수는 미결수들이 구금된 구치소가 아닌 일반 교도소에 구금되지만, 최씨의 경...",0
3,그리고 주권자 국민의 관심이 각별하게 필요한 시간'이라고 강조하며 국민들의 관심을 ...,0
4,"박 최고위원은 '독일이 2015년 최저임금제를 도입하고, 미국도 최저임금제를 도입하...",0
...,...,...
199995,신세계건설은 지난달 26일 사업목적을 기존의 건설업 및 건물관리 외에 복합쇼핑몰의 ...,1
199996,카메라 모듈의 ois 및 무선 충전 기능 채택으로 수익성 개선도 기대된다..,1
199997,"삼성전기의 투자의견 '매수'를 유지하고, 12개월 목표주가는 기존 8만5000원에서...",1
199998,"지난해는 춘절이 1월에 있어 지난 2013년 12월에 판매가 집중됐으나, 올해는 춘...",1


In [ ]:
data['label'].value_counts() # 데이터 불균형 체크

1    99978
0    99969
Name: label, dtype: int64

# Input for BERTDataset

In [ ]:
data_list = []

for sentence, label in zip(data['sentence'], data['label']):
  data = []
  data.append(sentence)
  data.append(label)
  data_list.append(data)

len(data_list)

199947

In [ ]:
data_list[:5]

[['도널드 트럼프 미국 대통령이 30일 역대 처음으로 우리나라 대통령과 비무장지대(dmz)를 동반 방문할 예정인 가운데 역대 어떤 미국 대통령이 dmz를 찾았는지 관심이 집중된다..',
  0],
 ["물 흐르듯 대화가 필요하다'고 작성했다..", 0],
 ['통상 기결수는 미결수들이 구금된 구치소가 아닌 일반 교도소에 구금되지만, 최씨의 경우엔 아직 대법원 재판이 남아 있어 구치소 생활을 계속할 전망이다.',
  0],
 ["그리고 주권자 국민의 관심이 각별하게 필요한 시간'이라고 강조하며 국민들의 관심을 촉구했다.", 0],
 ["박 최고위원은 '독일이 2015년 최저임금제를 도입하고, 미국도 최저임금제를 도입하고 확대했다.", 0]]

# Train, Test Split

In [ ]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32)

In [ ]:
print(len(dataset_train))
dataset_train[:5]

159957


[['유럽증시에서 도이치뱅크는 7.5% 급락한 10.55유로를 기록, 사상 최저치로 떨어졌다.', 1],
 ['현재 스위스와 터키, 우크라이나 등 25개국이 서머타임제도를 시행하는 것으로 알려졌다..', 0],
 ['미국은 이에 대해 군사행동을 포함한 모든 옵션을 고려하고 있다며 반군 압박에 나섰다.', 1],
 ['voting for the fomc monetary policy action were: janet l.', 0],
 ['개인이 1270억원 순매수했으며 외국인과 기관이 각각 494억원, 816억원 순매도했다.', 0]]

In [ ]:
print(len(dataset_test))
dataset_test[:5]

39990


[["다만 그는 '(바른미래당 입당을) 출마를 위해서 하는 것은 아니다.", 0],
 ["인수한 아이리버에 대해서는 '하이앤드 음향 시장에서 입지를 공고히 하고 있다'며 '해외시장과 앱세서리 등으로 영역을 확장하고 있다'고 설명했다..",
  1],
 ['20% 요금할인보다 지원금을 받는 것이 더 유리한 것처럼 속이거나 요금할인 대상이 아니라고 거짓 안내를 한 정확이 있었다는 것..',
  0],
 ['전문가들은 3월 엠파이어스테이트지수는 6.5를 기록해 전달의 4.5를 웃돌 것으로 예상하고 있다.', 1],
 ["빅앤트는 콘돔을 연상할 때 '부끄러운 생각' 대신 '바른생각'을 하자는 취지로 이번 캠페인을 진행했다고 밝혔다.", 0]]

## KoBERT tokenizer, model

[CLS] : classification token (문장의 시작)

[SEP] : seperate (문장 구분)

[MASK] : MASK

[PAD] : PADDING

# Dataset

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab = vocab, pad = pad, pair = pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
import torch
from transformers import BertModel
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


# Parameter

In [ ]:
# parameter 값 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3 # epoch = 4부터 overfitting 발생하므로 epoch = 3로 조절
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [ ]:
data_train.labels[:3]

[1, 0, 1]

In [ ]:
data_test.sentences[:3]

[(array([   2, 1569, 1191,  520,   18, 6273, 6115, 6258, 5804, 3836, 5804,
         7088,   40, 4568, 6141, 6116, 3568, 4930,  910, 3100,   54,    3,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
  array(22, dtype=int32),
  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=int32)),
 (array([   2, 3777, 7828, 3121, 6122, 6323, 6896, 1687,  520, 7782, 7096,
         6837, 5920, 3606, 7886, 2993, 3849, 6116, 1023, 5439, 7996, 4926,
         3862,   15, 6197,  520, 7848, 6995, 6716, 5468, 3205, 6579, 6553,
         6122, 1822, 3387, 7088, 5133, 7788, 3862,   

# DataLoader

In [ ]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# KoBERT 모델

In [ ]:
# KoBERT 오픈소스 내 예제코드 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2,
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

# optimizer and schedule

In [ ]:
# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
# 최적화해야 할 parameter를 optimizer에게 알려야 함
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate) # optimizer
loss_fn = nn.CrossEntropyLoss() # 분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Calculate Accuracy

In [ ]:
# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

# Train

epoch = 4 일 때, Overfitting 발생

In [ ]:
train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # train_history.append(train_acc / (batch_id+1))

    # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

<ipython-input-28-61774693c4e7>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/2500 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 2.4969303607940674 train acc 0.03125
epoch 1 batch id 201 loss 0.5106567740440369 train acc 0.5045864427860697
epoch 1 batch id 401 loss 0.17057935893535614 train acc 0.686175187032419
epoch 1 batch id 601 loss 0.25955355167388916 train acc 0.7543157237936772
epoch 1 batch id 801 loss 0.29868489503860474 train acc 0.7930711610486891
epoch 1 batch id 1001 loss 0.2925800681114197 train acc 0.8180881618381618
epoch 1 batch id 1201 loss 0.17477542161941528 train acc 0.836334304746045
epoch 1 batch id 1401 loss 0.17344747483730316 train acc 0.8500289971448965
epoch 1 batch id 1601 loss 0.10706518590450287 train acc 0.8606534978138664
epoch 1 batch id 1801 loss 0.07263337820768356 train acc 0.8698813159355914
epoch 1 batch id 2001 loss 0.05969781056046486 train acc 0.8772488755622189
epoch 1 batch id 2201 loss 0.115476593375206 train acc 0.8838880054520672
epoch 1 batch id 2401 loss 0.16746684908866882 train acc 0.889369012911287
epoch 1 train acc 0.8917681547619047


<ipython-input-28-61774693c4e7>:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

epoch 3 test acc 0.97165


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.01878465712070465 train acc 0.984375
epoch 4 batch id 201 loss 0.0006348067545332015 train acc 0.9951026119402985
epoch 4 batch id 401 loss 0.0007969684083946049 train acc 0.9946228179551122
epoch 4 batch id 601 loss 0.0003289966844022274 train acc 0.9945663477537438
epoch 4 batch id 801 loss 0.0009600779158063233 train acc 0.9947331460674157
epoch 4 batch id 1001 loss 0.001704991445876658 train acc 0.9949893856143857
epoch 4 batch id 1201 loss 0.00034523330396041274 train acc 0.9954204829308909
epoch 4 batch id 1401 loss 0.025217359885573387 train acc 0.9956392755174875
epoch 4 batch id 1601 loss 0.00040865407208912075 train acc 0.9958326826983136
epoch 4 batch id 1801 loss 0.0005116903339512646 train acc 0.9959137284841755
epoch 4 batch id 2001 loss 0.000380113982828334 train acc 0.9960488505747126
epoch 4 batch id 2201 loss 0.000273306475719437 train acc 0.9962233075874602
epoch 4 batch id 2401 loss 0.0015348196029663086 train acc 0.9963491774260724
epoch 4

  0%|          | 0/625 [00:00<?, ?it/s]

epoch 4 test acc 0.9712953703703704


epoch = 3일 때, 코드

In [ ]:
train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # train_history.append(train_acc / (batch_id+1))

    # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

<ipython-input-24-61774693c4e7>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/2500 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 0.7748388051986694 train acc 0.484375
epoch 1 batch id 201 loss 0.2951398491859436 train acc 0.7018812189054726
epoch 1 batch id 401 loss 0.1625792384147644 train acc 0.7840944513715711
epoch 1 batch id 601 loss 0.22319380939006805 train acc 0.8192595673876872
epoch 1 batch id 801 loss 0.25565722584724426 train acc 0.8419943820224719
epoch 1 batch id 1001 loss 0.20736975967884064 train acc 0.8568931068931069
epoch 1 batch id 1201 loss 0.15851666033267975 train acc 0.8700301831806828
epoch 1 batch id 1401 loss 0.15079626441001892 train acc 0.8797733761598858
epoch 1 batch id 1601 loss 0.08159391582012177 train acc 0.887326280449719
epoch 1 batch id 1801 loss 0.06611886620521545 train acc 0.8941560244308717
epoch 1 batch id 2001 loss 0.09407082200050354 train acc 0.8992691154422788
epoch 1 batch id 2201 loss 0.08547176420688629 train acc 0.9042693661971831
epoch 1 batch id 2401 loss 0.15820856392383575 train acc 0.908182788421491
epoch 1 train acc 0.91003720238095

<ipython-input-24-61774693c4e7>:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

epoch 2 test acc 0.96825


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.11505965888500214 train acc 0.96875
epoch 3 batch id 201 loss 0.004082473926246166 train acc 0.9856965174129353
epoch 3 batch id 401 loss 0.003129742806777358 train acc 0.9862453241895262
epoch 3 batch id 601 loss 0.015659324824810028 train acc 0.986714850249584
epoch 3 batch id 801 loss 0.012329705990850925 train acc 0.9879057428214731
epoch 3 batch id 1001 loss 0.0035361782647669315 train acc 0.988449050949051
epoch 3 batch id 1201 loss 0.001968528376892209 train acc 0.9892797668609492
epoch 3 batch id 1401 loss 0.011853467673063278 train acc 0.9898509992862241
epoch 3 batch id 1601 loss 0.001222530729137361 train acc 0.9902892723297939
epoch 3 batch id 1801 loss 0.0009378963150084019 train acc 0.990864450305386
epoch 3 batch id 2001 loss 0.0029941846150904894 train acc 0.9912777986006996
epoch 3 batch id 2201 loss 0.001073442748747766 train acc 0.991608927760109
epoch 3 batch id 2401 loss 0.03357130289077759 train acc 0.9917872761349438
epoch 3 train acc 0.

  0%|          | 0/625 [00:00<?, ?it/s]

epoch 3 test acc 0.9712453703703704


# 모델 저장

In [ ]:
PATH = '/content/drive/MyDrive/Colab_Notebooks/NewsBigData/'
# torch.save(model, PATH + 'hard_easy_classification_e5')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'hard_easy_classification_e3_state_dict.pt')  # 모델 객체의 state_dict 저장
# torch.save({
#     'model': model.state_dict(),
#     'optimizer': optimizer.state_dict()
# }, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

# Input Test

In [ ]:
#https://github.com/SKTBrain/KoBERT 의 파일들을 Colab으로 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install transformers
!pip3 install kobert-transformers
!pip install gluonnlp
!pip install sentencepiece
!pip install mxnet
!pip install torch

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-8dv4109j/kobert-tokenizer_fd938ef782fe4d25beb3cfdd45cf152f
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-8dv4109j/kobert-tokenizer_fd938ef782fe4d25beb3cfdd45cf152f
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=4c0e02afd365a3541546198d1d641303ec0d68fa903a69ee6cbdb389ef54bd0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-ss564fve/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 118.0 MB/s e

In [ ]:
import gluonnlp as nlp
from torch.utils.data import Dataset
import numpy as np
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab = vocab, pad = pad, pair = pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [ ]:
import torch
from torch import nn

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2,
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [ ]:
import torch
import gluonnlp as nlp
import numpy as np
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from kss import split_sentences

In [ ]:
# parameter
max_len = 128
batch_size = 32

# device
device = torch.device('cpu')

#BERT 모델, Vocabulary 불러오기, 토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

## 학습 모델 불러오기
PATH = '/content/drive/MyDrive/Colab_Notebooks/NewsBigData/hard_easy_classification_e3_state_dict.pt'
model = BERTClassifier(bert=bertmodel)
model.load_state_dict(torch.load(PATH, map_location=device))
model.eval()


def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=4)

    model.eval()

    answer = 0

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for logits in out:
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("쉬운")
            else:
                test_eval.append("어려운")

        return test_eval[0]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
text = '''구광모 LG그룹 회장이 LG트윈스가 29년 만에 KBO 한국시리즈 우승을 차지한 것과 관련해 '통 큰 할인' 이벤트로 응원을 해 준 팬들에게 보답한다. LG전자 TV·냉장고 등 가전제품 할인율은 29년만에 한국시리즈를 제패한 것에서 착안해 29%로 정해졌다.
16일 LG그룹에 따르면 LG전자, LG생활건강, LG유플러스, 곤지암리조트 등 일부 계열사들은 한국 시리즈에서 LG트윈스가 우승을 차지한 것과 관련해 초대형 할인 행사에 돌입한다.

LG전자는 'LG 윈윈 페스티벌'을 통해 주요 가전 제품을 29% 할인하고 경품 추천 이벤트를 진행한다. 오는 21일 온라인 브랜드숍에서 오전 10시부터 특정 행사 모델을 대상으로 한정 수량 29% 할인 판매하는 것이 이번 행사의 핵심이다.

LG전자 관계자는 "구체적인 할인 품목은 21일 당일 LG전자의 온라인 브랜드숍에 공지가 될 것"이라며 "29%란 할인율은 임직원 할인율과 비교해도 상당히 파격적인 수준"이라고 설명했다.

18일부터는 소진 시까지 LG트윈스 챔피언 에디션으로 특별 제작한 스탠바이미고(Go) 한정판을 온라인 브랜드숍에서 판매한다. 스탠바이미고(Go)에 정규 시리즈 우승 엠블럼을 디지털 인쇄한 제품이다.

LG베스트샵 매장 방문 인증 이벤트도 진행한다. 전국 베스트샵 지점의 행사 포스터 앞에서 인증샷을 촬영, 축하 멘트 및 해시태그와 함께 SNS에 올리면 3500명을 추첨해 LG 스탠바이미고(Go), LG 엑스붐 360, LG트윈스 유광 점퍼, 커피 쿠폰 등을 선물한다.

생활용품업체인 LG생활건강도 뷰티, 생활용품 온라인 직영몰을 중심으로 각 브랜드의 대표 제품들을 소비자가에서 71% 할인된 '29%' 가격에 판매키로 했다. 이에 따라 화장품 '오휘'의 6만원짜리 '얼티밋 핏 톤업 진 쿠션'은 1만7400원에, '빌리프'의 2만9000원짜리 '더 트루 크림 아쿠아밤 (30mL)'는 8410원에 판매된다.

통신사 LG유플러스는 오는 17일부터 26일까지 멤버스 앱을 통해 응모하면 OTT 1년 사용권, LG트윈스 유광점퍼, LG트윈스 콜라보레이션 야구모자, 편의점 음료쿠폰를 추첨 증정한다.

데이터커머스 플랫폼 'U+콕'에서는 오는 29일부터 다음 달 27일까지 29일 동안 매일 선착순 1000명(총 2만9000명)에게 10% 할인쿠폰을 증정하기로 했다. 12월 한 달 동안은 LG전자와 LG생활건강의 제품을 2900원, 2만9000원, 29만원에 한정 판매하는 '트윈스 프라이데이' 행사도 진행한다.

경기도 광주 곤지암리조트는 오는 17일부터 26일까지 LG트윈스와 협업해 LG트윈스 캐릭터와 인기 선수 사인 및 다양한 굿즈를 놓아둔 테마룸인 '트윈스룸'을 50% 할인 요금에 제공한다. 트윈스 슬리퍼와 응원 타월도 제공한다. 곤지암리조트 공식 홈페이지와 숙박 예약 플랫폼 야놀자 등을 통해 예약할 수 있다.

LG아트센터 서울도 17~19일 3일간 '클럽 아크 위드 안테나'와 연극 '튜링머신' 등 2편의 공연 티켓을 29% 할인 판매한다.

이 외에 LG트윈스 유니폼 제작업체인 프로스펙스에서는 지난 15일부터 이번 우승 기념 티셔츠 등을 최대 30% 할인해 한정 판매하고 있다.

인터파크의 온라인 트윈스숍에서는 어센틱 우승모자, 티셔츠, 후드티, 와펜 등 LG트윈스 우승 굿즈를 지난 15일부터 판매했다. 하지만 판매와 동시에 과도한 접속량으로 인해 서버 증설 작업을 해야만 했다.

신한카드 역시 16일 LG트윈스 우승 엠블럼이 각인된 신용카드를 2900명에게만 발급해주는 이벤트를 진행한다.

이처럼 LG그룹이 대대적 할인 행사를 준비한 것은 그간 한국시리즈에서 우승을 차지한 야구팀의 모기업이 시즌 내내 구단을 응원해 준 팬들을 위해 다양한 보답 행사를 펼쳐왔기 때문이다. 실제 2014년 삼성라이온즈가 통산 8번째 한국시리즈 우승을 하자 삼성전자는 'UHD(초고선명도) TV' 최대 111만원 할인 혜택 이벤트를, 기아는 2017년 기아타이거즈가 우승하자 선착순 1만1000명에게 모닝과 레이 등 자동차를 최대 12% 인하한 가격에 판매했다.

구 회장은 LG트윈스 팬들을 위한 이벤트뿐 아니라 선수단에도 통 크게 포상했다. 구단은 KBO가 주는 우승 배당금 29억4000만원에 모기업인 LG가 주는 우승 보너스까지 더해 최대 44억1000만원을 받을 것으로 관측된다.

또 구 회장은 오는 17일 LG트윈스의 우승을 축하하는 '축승회(祝勝會)'에도 참석할 예정이다. 서울 강서구 마곡동의 LG사이언스파크에서 비공개로 열리는 축승회에서 구 회장은 고(故) 구본무 선대회장이 직접 마련한 롤렉스 시계 전달식을 진행하고, 아와모리 소주도 봉인 해제해 선수들과 맛볼 것으로 알려졌다.

아와모리 소주는 1994년 우승 축승회에서 LG 선수단과 관계자들이 축배를 들었던 술이다. 당시 구 선대회장은 "내년에도 우승하면 이 술로 다시 건배하자"라고 말한 바 있다. 이에 이듬해 전지훈련을 마치고 돌아오면서 3병을 사 왔으나 30년간 축배를 들지 못했다. 롤렉스 시계는 주장 오지환 선수에게 지급됐으나, 오 선수가 "선대회장님의 유품이나 마찬가지라서 구광모 회장님께 드리고 누구나 볼 수 있도록 전시했으면 한다"고 밝혀 눈길을 끌었다.

구본무 선대회장만큼 '야구광'으로 알려진 구 회장은 지난 2018년 취임한 후 처음으로 이번 한국시리즈 경기를 관람했다. 그는 1·4·5차전까지 세 차례나 직관(직접 관람)할 정도로 이번 경기에 관심을 기울였다.

또 구 회장은 우승을 차지한 날 직접 마이크를 잡고 "너무나 감격스럽다. 세계 최고의 무적 LG 팬 여러분"이라고 운을 뗀 뒤 "LG트윈스가 29년만에 드디어 우승했다. 29년이라는 오랜 기다림 속에서도 변함없이 LG트윈스를 사랑하고 응원해주신 팬 여러분들께 감사드린다"고 언급했다.

그러면서 "매 순간 최고의 감동을 선사해준 우리 자랑스러운 선수들과 스텝 여러분들께도 진심으로 감사드리고 축하드린다"며 "오늘의 승리는 여기 계신 모든 분들과 LG트윈스를 사랑해 주시는 모든 분들이 함께 일궈낸 값진 승리다. 2023년 챔피언은 LG트윈스입니다. 무적 LG 파이팅입니다"라고 외치며 소감을 마무리했다.'''


In [ ]:
sentences = split_sentences(text)

For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://github.com/hyunwoongko/python-mecab-kor
- konlpy.tag.Mecab: https://konlpy.org/en/latest/api/konlpy.tag/#mecab-class



In [ ]:
sl = []
for sentence in sentences:
    #TODO: if the result of .predict() is '어려운 문장'
    result = predict(sentence)
    if(result =='어려운'):
        sl.append(sentence)
print(sl)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


["구광모 LG그룹 회장이 LG트윈스가 29년 만에 KBO 한국시리즈 우승을 차지한 것과 관련해 '통 큰 할인' 이벤트로 응원을 해 준 팬들에게 보답한다.", '16일 LG그룹에 따르면 LG전자, LG생활건강, LG유플러스, 곤지암리조트 등 일부 계열사들은 한국 시리즈에서 LG트윈스가 우승을 차지한 것과 관련해 초대형 할인 행사에 돌입한다.', "LG전자는 'LG 윈윈 페스티벌'을 통해 주요 가전 제품을 29% 할인하고 경품 추천 이벤트를 진행한다.", 'LG전자 관계자는 "구체적인 할인 품목은 21일 당일 LG전자의 온라인 브랜드숍에 공지가 될 것"이라며 "29%란 할인율은 임직원 할인율과 비교해도 상당히 파격적인 수준"이라고 설명했다.', '18일부터는 소진 시까지 LG트윈스 챔피언 에디션으로 특별 제작한 스탠바이미고(Go) 한정판을 온라인 브랜드숍에서 판매한다.', '스탠바이미고(Go)에 정규 시리즈 우승 엠블럼을 디지털 인쇄한 제품이다.', '전국 베스트샵 지점의 행사 포스터 앞에서 인증샷을 촬영, 축하 멘트 및 해시태그와 함께 SNS에 올리면 3500명을 추첨해 LG 스탠바이미고(Go), LG 엑스붐 360, LG트윈스 유광 점퍼, 커피 쿠폰 등을 선물한다.', "이에 따라 화장품 '오휘'의 6만원짜리 '얼티밋 핏 톤업 진 쿠션'은 1만7400원에, '빌리프'의 2만9000원짜리 '더 트루 크림 아쿠아밤 (30mL)'는 8410원에 판매된다.", '통신사 LG유플러스는 오는 17일부터 26일까지 멤버스 앱을 통해 응모하면 OTT 1년 사용권, LG트윈스 유광점퍼, LG트윈스 콜라보레이션 야구모자, 편의점 음료쿠폰를 추첨 증정한다.', "데이터커머스 플랫폼 'U+콕'에서는 오는 29일부터 다음 달 27일까지 29일 동안 매일 선착순 1000명(총 2만9000명)에게 10% 할인쿠폰을 증정하기로 했다.", "12월 한 달 동안은 LG전자와 LG생활건강의 제품을 2900원, 2만9000원, 29만원에 한정 판매하는 '트윈스 프라이데이' 행사도 진행한다.",

In [ ]:
print(len(sentences))
print(len(sl))

37
23


In [ ]:
while True:
    sentence = input("문장 난이도(쉬움/어려움)를 판단할 문장을 입력해주세요 : ")
    if sentence == "0":
      print(">> 난이도 판단을 종료합니다!")
      break
    print(">> 해당 문장은 " + predict(sentence) + " 문장 입니다.")
    print("\n")

문장 난이도(쉬움/어려움)를 판단할 문장을 입력해주세요 : 정부가 어제 발표한 국내 증시 모든 종목 공매도 금지 조치가 오늘부터 시행됐습니다.


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 해당 문장은 어려운 문장 입니다.


문장 난이도(쉬움/어려움)를 판단할 문장을 입력해주세요 : 공매도 중단의 영향으로 오늘 주식시장에선 급등세가 나타났습니다.
>> 해당 문장은 쉬운 문장 입니다.


문장 난이도(쉬움/어려움)를 판단할 문장을 입력해주세요 : 코스피는 큰 폭으로 상승하며 단숨에 2,500대로 올라섰습니다.
>> 해당 문장은 쉬운 문장 입니다.


문장 난이도(쉬움/어려움)를 판단할 문장을 입력해주세요 : 박진수 기자가 전해드립니다.
>> 해당 문장은 쉬운 문장 입니다.


문장 난이도(쉬움/어려움)를 판단할 문장을 입력해주세요 : 공매도 금지 첫날, 주가는 크게 뛰었습니다
>> 해당 문장은 쉬운 문장 입니다.


문장 난이도(쉬움/어려움)를 판단할 문장을 입력해주세요 : 코스피는 전 거래일보다 5.66%, 130포인트 넘게 올라 2,502.37로 마감했습니다.
>> 해당 문장은 쉬운 문장 입니다.


문장 난이도(쉬움/어려움)를 판단할 문장을 입력해주세요 : 코스닥지수는 7.34% 상승해 840선에 바짝 다가섰습니다.
>> 해당 문장은 쉬운 문장 입니다.


문장 난이도(쉬움/어려움)를 판단할 문장을 입력해주세요 : 지수가 급등하면서 코스닥 시장에선 오전 9시 57분쯤 프로그램 매수 호가 일시 효력 정지, 이른바 사이드카가 발동되기도 했습니다.
>> 해당 문장은 어려운 문장 입니다.


문장 난이도(쉬움/어려움)를 판단할 문장을 입력해주세요 : 2020년 6월 이후 처음입니다.
>> 해당 문장은 쉬운 문장 입니다.


문장 난이도(쉬움/어려움)를 판단할 문장을 입력해주세요 : 시장이 이렇게 들썩인 건 어제 정부가 전격적으로 발표한 공매도 전면 금지 조치의 영향으로 보입니다.
>> 해당 문장은 어려운 문장 입니다.


문장 난이도(쉬움/어려움)를 판단할 문장을 입력해주세요 : 공매도는 특정 기업의 주가 하락을 예상해서 가지고 있지 않은 주식을 빌려 팔았다가 주가가 내려가면 싸게 사서 되갚아 차익을 얻는 투자 기법입니다.
>> 해당 문장은 